In [1]:
number_of_terminals = int(input("Enter the number of terminals : "))
terminals = []
print("Enter the terminals : ")
for i in range(number_of_terminals):
    terminals.append(input())

Enter the terminals : 


In [2]:
number_of_nonterminals = int(input("Enter the number of non terminals : "))
nonterminals = []
print("Enter the non terminals : ")
for j in range(number_of_nonterminals):
    nonterminals.append(input())


Enter the non terminals : 


In [3]:
start_symbol = input("Enter the start symbol : ")
production_count = int(input("Enter the number of productions : "))
productions = []
print("Enter the productions : ")
for k in range(production_count):
    productions.append(input())

    
'''
S->NP VP
NP->P
NP->PN
NP->DN
VP->V NP
N->Championship
N->ball
N->toss
V->is
V->want
V->won
V->played
P->me
P->I
P->you
PN->India
PN->Australia
PN->Steve
PN->John
D->the
D->a
D->an
'''

Enter the productions : 


'\nS->NP VP\nNP->P\nNP->PN\nNP->DN\nVP->V NP\nN->Championship\nN->ball\nN->toss\nV->is\nV->want\nV->won\nV->played\nP->me\nP->I\nP->you\nPN->India\nPN->Australia\nPN->Steve\nPN->John\nD->the\nD->a\nD->an\n'

In [4]:
# input testing --------(remove while cleaning)
print("terminals : ",terminals)
print("non terminals : ",nonterminals)
print("productions : ",productions)

terminals :  ['a', 'b', 'p', 'c']
non terminals :  ['S', 'A', 'B', 'C']
productions :  ['S->ABC', 'S->C', 'A->a', 'A->bB', 'A->#', 'B->p', 'B->#', 'C->c']


In [5]:
production_dict = {}
for nt in nonterminals:
    production_dict[nt] = []

# split the productions into parts to simplify parsing
for production in productions:
    nonterminal_to_production = production.split("->")
    expanded = nonterminal_to_production[1].split("/") # assumption : single char terminals
    for ex in expanded:
        production_dict[nonterminal_to_production[0]].append(ex)


In [6]:
print("production_dict",production_dict)

production_dict {'S': ['ABC', 'C'], 'A': ['a', 'bB', '#'], 'B': ['p', '#'], 'C': ['c']}


In [7]:
# declare dicts for first and follow as they are set of elements mapped to keys(non terminals)
FIRST = {}
FOLLOW = {}

In [8]:
def First(string):
    first_ = set()
    
    if string in nonterminals:
        alternatives = production_dict[string]

        for alternative in alternatives:
            first_2 = First(alternative)
            first_ = first_ | first_2

    elif string in terminals:
        first_ = {string}

    elif string == '' or string == '#':
        first_ = {'#'}

    else:
        first_2 = First(string[0])
        if '#' in first_2:
            i = 1
            while '#' in first_2:
                

                first_ = first_ | (first_2 - {'#'})
                
                if string[i:] in terminals:
                    first_ = first_ | {string[i:]}
                    break
                elif string[i:] == '':
                    first_ = first_ | {'#'}
                    break
                first_2 = First(string[i:])
                first_ = first_ | first_2 - {'#'}
                i += 1
        else:
            first_ = first_ | first_2

    
    return first_



In [9]:
# create set for each Non terminal for First and Follow
for nonterminal in nonterminals:
    FIRST[nonterminal] = set()
    FOLLOW[nonterminal] = set()

for nonterminal in nonterminals:
    FIRST[nonterminal] = FIRST[nonterminal] | First(nonterminal)

In [10]:
def Follow(nT):
    
    follow_ = set()
    
    prods = production_dict.items()
    if nT == start_symbol:
        follow_ = follow_ | {'$'}
    for nt, rhs in prods:
        
        for alt in rhs:
            for char in alt:
                if char == nT:
                    following_str = alt[alt.index(char) + 1:]
                    if following_str == '':
                        if nt == nT:
                            continue
                        else:
                            follow_ = follow_ | Follow(nt)
                    else:
                        follow_2 = First(following_str)
                        if '#' in follow_2:
                            follow_ = follow_ | follow_2-{'#'}
                            follow_ = follow_ | Follow(nt)
                        else:
                            follow_ = follow_ | follow_2
    
    return follow_



In [11]:
FOLLOW[start_symbol]=FOLLOW[start_symbol] | {'$'}
for non_terminal in nonterminals:
    FOLLOW[non_terminal] = FOLLOW[non_terminal] | Follow(non_terminal)

In [12]:
for non_terminal in nonterminals:
    print("first(",non_terminal,") ==> ",FIRST[non_terminal])

first( S ) ==>  {'p', 'a', 'b', 'c'}
first( A ) ==>  {'a', 'b', '#'}
first( B ) ==>  {'p', '#'}
first( C ) ==>  {'c'}


In [13]:
for non_terminal in nonterminals:
    print("follow(",non_terminal,") ==> ",FOLLOW[non_terminal])

follow( S ) ==>  {'$'}
follow( A ) ==>  {'p', 'c'}
follow( B ) ==>  {'p', 'c'}
follow( C ) ==>  {'$'}


In [14]:
import re
import string
import pandas as pd 



In [15]:
def ll1(follow, productions):

    print("\nParsing Table\n")

    table = {}
    for key in productions:
        for value in productions[key]:
            if value!='@':
                for element in First(value):
                    table[key, element] = value
            else:
                for element in follow[key]:
                    table[key, element] = value

    for key,val in table.items():
        print(key,"=>",val)

    new_table = {}
    for pair in table:
        new_table[pair[1]] = {}

    for pair in table:
        new_table[pair[1]][pair[0]] = table[pair]


    print("\n")
    print("\nParsing Table in matrix form\n")
    print (pd.DataFrame(new_table).fillna('-'))
    print ("\n")

    return table

ll1Table = ll1(FOLLOW, production_dict)

print(ll1Table)



Parsing Table

('S', 'p') => ABC
('S', 'a') => ABC
('S', 'b') => ABC
('S', 'c') => C
('A', 'a') => a
('A', 'b') => bB
('A', '#') => #
('B', 'p') => p
('B', '#') => #
('C', 'c') => c



Parsing Table in matrix form

     p    a    b  c  #
S  ABC  ABC  ABC  C  -
B    p    -    -  -  #
A    -    a   bB  -  #
C    -    -    -  c  -


{('S', 'p'): 'ABC', ('S', 'a'): 'ABC', ('S', 'b'): 'ABC', ('S', 'c'): 'C', ('A', 'a'): 'a', ('A', 'b'): 'bB', ('A', '#'): '#', ('B', 'p'): 'p', ('B', '#'): '#', ('C', 'c'): 'c'}


In [16]:
def parse(user_input,start_symbol,parsingTable):

    #flag for checking string is accepted or not.
    flag = 0

    #append dollar at the end of string
    user_input = user_input + "$"

    #empty stack
    stack = []
    
    #initially stack contain Start symbol follow by dollar symbol
    stack.append("$")
    stack.append(start_symbol)

    input_len = len(user_input)
    buffer = 0

    #until stack is not empty loop is going on
    while len(stack) > 0:

        #get top index from length of stack
        top = stack[len(stack)-1]

        print("Top =>",top)

        #current input
        current_input = user_input[buffer]

        print("Current_Input => ",current_input)

        if top == current_input:
            stack.pop()
            buffer = buffer + 1
        else:
            
            # As LL1 Table contain key as combination of (terminal (row_name), non terminal (column_name))
            #finding value for key in table
            key = top , current_input
            print(key)

            #top of stack terminal => not accepted
            if key not in parsingTable:
                flag = 1
                break

            value = parsingTable[key]
            if value !='#':
                
                # reverse the extracted value from table 
                value = value[::-1]
                value = list(value)
                
                #poping top of stack
                stack.pop()

                #push value chars to stack
                for element in value:
                    stack.append(element)
            else:
                stack.pop()		

    if flag == 0:
        print("String accepted!")
    else:
        print("String not accepted!")



In [17]:
start = input("Enter the Start Symbol : ")
parse("apc",start,ll1Table)


Top => S
Current_Input =>  a
('S', 'a')
Top => A
Current_Input =>  a
('A', 'a')
Top => a
Current_Input =>  a
Top => B
Current_Input =>  p
('B', 'p')
Top => p
Current_Input =>  p
Top => C
Current_Input =>  c
('C', 'c')
Top => c
Current_Input =>  c
Top => $
Current_Input =>  $
String accepted!
